<a href="https://colab.research.google.com/github/Kebrenok/Simulative-/blob/main/%D0%91%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82_%22%D0%9F%D0%B5%D1%80%D0%B2%D1%8B%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81%22_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%87%D0%B5%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 кейс

**Ваша задача написать функцию `process_files`, которая принимает на вход два пути к папкам. Из первой папки необходимо выбрать все "чеки" (файлы по шаблону из условия), а во вторую папку сохранить один объединенный чек (отсортированный по дате, а затем по продукту) под названием `combined_data.csv`.**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить папку с файлами. После выполнения, в папке `reports_main` будут храниться все присланные магазинами чеки.

In [3]:
!wget https://github.com/vs8th/reports/archive/main.zip

import zipfile

with zipfile.ZipFile("main.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

!rm main.zip

--2025-06-25 15:26:39--  https://github.com/vs8th/reports/archive/main.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Vs8th/reports/zip/refs/heads/main [following]
--2025-06-25 15:26:39--  https://codeload.github.com/Vs8th/reports/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ]   3.22K  --.-KB/s    in 0s      

2025-06-25 15:26:39 (47.4 MB/s) - ‘main.zip’ saved [3296]



Чтобы посмотреть как выглядят подходящие для объединения чеки выполните следующую ячейку.

In [ ]:
import pandas as pd

df = pd.read_csv('reports-main/2023-02-17-05-38-2.csv', sep=";")
df

,date,product,store,cost
0,2023-02-17,product_0,store_2,10
1,2023-02-17,product_1,store_2,20
2,2023-02-17,product_2,store_2,30
3,2023-02-17,product_3,store_2,40
4,2023-02-17,product_4,store_2,50


**Решение**

Напишите свое решение ниже

**Примечание**

Не все файлы подходящие по названию, будут подходить по содержанию. Там может оказаться лишний столбец, например. Ориентируйтесь на столбцы из чека выше - это то, что вас интересует. Остальные столбцы можно просто отбросить.

**Важно**: разделителем файла на выходе должна быть запятая.

In [25]:
import csv
import glob
import re
import os
#импортируем нужные модули

def process_files(src_folder, dest_folder):

  expected_header = ['date', 'product', 'store', 'cost']
  pattern1 = r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d+.csv'
  lst_files=glob.glob(src_folder + '/*.csv')
  data=[]
  sort_columns = (0, 1)
#задаем шаблон проверки заголока, шаблон проверки названия файла, формируем список файлов в директории, создаем список данных, задаем кортеж столбцов сортировки
  for item in lst_files:
    file_name = os.path.basename(item) #отсекаем путь файла, только название и расширение
    match = re.match(pattern1, file_name) # проверяем имя файла на соответствие шаблону
    if match:
      # если файл соответствует по названию заданному шаблону, проверяем сответствие заголовков
      with open(item, 'r', newline='') as bill:
        reader=csv.reader(bill, delimiter=';')
        header = next(reader)
        if header ==expected_header:
          for row in reader:
            data.append(row)
        else:
          #если есть заголовок, которого нет в целевых, удаляем ст=олбец под этим заголовком, остальное - пишем в список
          for head in header:
            if head not in expected_header:
              col_to_del=header.index(head)
              for row in reader:
                del row[col_to_del]
                data.append(row)
                #сортируем данные по 2м столбцам
  data_sorted = sorted(data, key=lambda row:(row[sort_columns[0]],row[sort_columns[1]]))
#создаем директорию для записи финального файла
  if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)
  output_file_path = os.path.join(dest_folder, 'combined_data.csv')
  with open(output_file_path, 'w', newline='') as cd:
    writer=csv.writer(cd, delimiter=',')
    writer.writerow(expected_header)
    for row in data_sorted:
      writer.writerow(row)


src_folder = 'reports-main'
dest_folder = 'comb_reports'
process_files(src_folder, dest_folder)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [26]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv

import pandas as pd

user_answer = pd.read_csv(f'{dest_folder}/combined_data.csv')
correct_answer = pd.read_csv('data.csv')

--2025-06-25 15:50:12--  https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv [following]
--2025-06-25 15:50:12--  https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984 [text/plain]
Saving to: ‘data.csv.1’

data.csv.1          100%[===================>]     984  --.-KB/s    in 0s      

2025-06-25 15:50:12 (42.6 MB/s) - ‘data.csv.1’ saved [984/984]



In [27]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!')

Поздравляем, Вы справились и успешно прошли все проверки!
